In [1]:
# ✅ Step 1: Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Dropout

# ✅ Step 2: Load Preprocessed Data
print("📥 Loading preprocessed dataset...")
df = pd.read_csv("network_data.csv") 
print(f"✅ Dataset loaded! Shape: {df.shape}")

📥 Loading preprocessed dataset...
✅ Dataset loaded! Shape: (2435692, 122)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

print("\u2705 Step 1: Libraries imported successfully!")

# Step 2: Extract Features and Target
features = [
    "duration", "packets_count", "fwd_packets_count", "bwd_packets_count",
    "total_payload_bytes", "fwd_total_payload_bytes", "bwd_total_payload_bytes",
    "payload_bytes_max", "payload_bytes_min", "payload_bytes_mean",
    "bytes_rate", "packets_rate",
    "fwd_total_header_bytes", "bwd_total_header_bytes",
    "avg_segment_size", "fwd_avg_segment_size", "bwd_avg_segment_size",
    "fwd_init_win_bytes", "bwd_init_win_bytes",
    "active_mean", "idle_mean", "down_up_rate"
]
target = "label"

print("\u2705 Step 2: Features and target selected.")

# Load dataset (Ensure df is defined before running this script)
X = df[features].copy()
y = df[target]

# Step 3: Handle Missing Values
X.fillna(X.mean(), inplace=True)
print("\u2705 Step 3: Missing values handled.")

# Step 4: Convert Target to Numeric Labels
le = LabelEncoder()
y = le.fit_transform(y)
print("\u2705 Step 4: Target labels encoded.")

# Step 5: Normalize Features
scaler = StandardScaler()
X = scaler.fit_transform(X)
print("\u2705 Step 5: Features normalized.")

# Step 6: Split Data into Train & Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\u2705 Step 6: Data split completed - Train: {X_train.shape}, Test: {X_test.shape}")

# Step 7: Build Optimized Model
print("\ud83d\ude80 Step 7: Building model...")
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(len(np.unique(y)), activation='softmax')
])
print("\u2705 Step 7: Model built successfully!")

# Step 8: Compile Model with Learning Rate Scheduling
lr_schedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("\u2705 Step 8: Model compiled.")

# Step 9: Train Model with Early Stopping
print("\ud83d\ude80 Step 9: Training model...")
history = model.fit(
    X_train, y_train, epochs=50, batch_size=64,
    validation_data=(X_test, y_test), callbacks=[lr_schedule, early_stopping]
)
print("\u2705 Step 9: Model training completed!")

# Step 10: Evaluate Model
print("\ud83d\ude80 Step 10: Evaluating model...")
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\u2705 Step 10: Test Accuracy: {test_acc:.4f}")

# Step 11: Save Model & Preprocessing Steps
model.save("network_packet_classifier_v2.h5")
print("\u2705 Step 11: Model saved as 'network_packet_classifier_v2.h5'")

# Step 12: Save Label Encoder & Scaler
import joblib
joblib.dump(le, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")
print("\u2705 Step 12: Preprocessing objects saved.")

print("\ud83c\udf89 Training complete! \ud83d\ude80")


In [ ]:
print("All Features in Dataset:")
print(df.columns.tolist())  # Prints all column names in the dataset


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

print("✅ Step 1: Importing required libraries...")

# Predict labels for test data
print("🔄 Step 2: Making predictions on the test set...")
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
print("✅ Step 2: Predictions completed!")

# Generate confusion matrix
print("🔄 Step 3: Generating confusion matrix...")
cm = confusion_matrix(y_test, y_pred_classes)
print("✅ Step 3: Confusion matrix generated!")

# Plot confusion matrix
print("📊 Step 4: Plotting confusion matrix...")
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
print("✅ Step 4: Confusion matrix displayed!")

# Print classification report
print("📜 Step 5: Generating classification report...")
print("\nClassification Report:\n", classification_report(y_test, y_pred_classes))
print("✅ Step 5: Classification report displayed!")

print("🎯 Confusion Matrix and Classification Report Process Completed Successfully! 🚀")
